We tried running several other models. Didn't work out since the best algorithm to deal with a huge number of features is the random forest

Tried to find correlation between the data by using covariance matrix. Didn't worked out.

Tried to treat the data as time series to see what we could derive from it

Tried PCA + RF

Tried to spot outliers

## Structure of the Dataset

### Main data

In [268]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()

In [269]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import mean_squared_error

In [270]:
problem = imp.load_source('', 'problem.py') #problem is a module with different methods
X_df, y_array = problem.get_train_data() #get_train_data() is one of those methods

X_train = X_df.copy()
y_train = y_array.copy()

In [271]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [272]:
X_train.describe()

,WeeksToDeparture,std_wtd
count,8902.000000,8902.000000
mean,11.446469,8.617773
std,2.787140,2.139604
min,2.625000,2.160247
25%,9.523810,7.089538
50%,11.300000,8.571116
75%,13.240000,10.140521
max,23.163265,15.862216


In [273]:
X_train.shape

(8902, 5)

In [274]:
pd.DataFrame(data=X_train.isna().sum(), columns=["Number of NAs"]).transpose()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
Number of NAs,0,0,0,0,0


There are no missing values

### Additional Data (IATA)

In [275]:
#Join iata data
iata = pd.read_csv('data/IATA_US_CODES.csv')
iata['IATA Code']=iata['IATA Code'].astype(str)
iata['IATA Code']=iata['IATA Code'].str.rstrip()
iata=iata.set_index('IATA Code')
iata_dep=iata.add_suffix('_Dep')
iata_arr=iata.add_suffix('_Arr')

In [276]:
X_train=X_train.join(iata_dep, on='Departure', how='left')
X_train=X_train.join(iata_arr, on='Arrival', how='left')
X_train

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Rank_2018_Dep,Airports (large hubs)_Dep,Major city served_Dep,State_Dep,2018_Dep,2017_Dep,2016_Dep,2015_Dep,Rank_2018_Arr,Airports (large hubs)_Arr,Major city served_Arr,State_Arr,2018_Arr,2017_Arr,2016_Arr,2015_Arr
0,2012-06-19,ORD,DFW,12.875000,9.812647,3,O'Hare International Airport,Chicago,IL,"39,874,879","38,593,028","37,589,899","36,305,668",4,Dallas/Fort Worth International Airport,Dallas/Fort Worth,TX,"32,800,721","31,816,933","31,283,579","31,589,832"
1,2012-09-10,LAS,DEN,14.285714,9.466734,9,McCarran International Airport,Las Vegas,NV,"23,655,285","23,364,393","22,833,267","21,824,231",5,Denver International Airport,Denver,CO,"31,363,573","29,809,097","28,267,394","26,280,043"
2,2012-10-05,DEN,LAX,10.863636,9.035883,5,Denver International Airport,Denver,CO,"31,363,573","29,809,097","28,267,394","26,280,043",2,Los Angeles International Airport,Los Angeles,CA,"42,626,783","41,232,432","39,636,042","36,351,226"
3,2011-10-09,ATL,ORD,11.480000,7.990202,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,"51,866,464","50,251,964","50,501,858","49,340,732",3,O'Hare International Airport,Chicago,IL,"39,874,879","38,593,028","37,589,899","36,305,668"
4,2012-02-21,DEN,SFO,11.450000,9.517159,5,Denver International Airport,Denver,CO,"31,363,573","29,809,097","28,267,394","26,280,043",7,San Francisco International Airport,San Francisco,CA,"27,794,154","26,900,048","25,707,101","24,190,549"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8897,2011-10-02,DTW,ATL,9.263158,7.316967,19,Detroit Metropolitan Airport,Detroit,MI,"17,437,716","17,036,092","16,826,287","16,255,520",1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,"51,866,464","50,251,964","50,501,858","49,340,732"
8898,2012-09-25,DFW,ORD,12.772727,10.641034,4,Dallas/Fort Worth International Airport,Dallas/Fort Worth,TX,"32,800,721","31,816,933","31,283,579","31,589,832",3,O'Hare International Airport,Chicago,IL,"39,874,879","38,593,028","37,589,899","36,305,668"
8899,2012-01-19,SFO,LAS,11.047619,7.908705,7,San Francisco International Airport,San Francisco,CA,"27,794,154","26,900,048","25,707,101","24,190,549",9,McCarran International Airport,Las Vegas,NV,"23,655,285","23,364,393","22,833,267","21,824,231"
8900,2013-02-03,ORD,PHL,6.076923,4.030334,3,O'Hare International Airport,Chicago,IL,"39,874,879","38,593,028","37,589,899","36,305,668",20,Philadelphia International Airport,Philadelphia,PA,"15,285,948","14,271,243","14,564,419","15,101,318"


In [283]:
#Add distance between cities
us_cities = pd.read_csv('data/uscities.csv')
us_cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500.0,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840116412
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947.0,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840097718
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441.0,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840096300
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507.0,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840097082
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591.0,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840116371


## Data cleaning and preprocessing

In [278]:
X_train = X_df.copy()
y_train = y_array.copy()

In [279]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [280]:
import os

def transform(X_df):
    X_encoded = X_df.copy()

    __file__ = 'submissions/starting_kit/'
    path = os.path.dirname(__file__)
    data_weather = pd.read_csv(os.path.join(path, 'external_data.csv'))
    X_weather = data_weather[['Date', 'AirPort', 'Max TemperatureC']]
    X_weather = X_weather.rename(
        columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

    #External data
    X_encoded = pd.merge(
        X_encoded, X_weather, how='left',
        left_on=['DateOfDeparture', 'Arrival'],
        right_on=['DateOfDeparture', 'Arrival'],
        sort=False)
    #Dummy Depart/Arr
    X_encoded = X_encoded.join(pd.get_dummies(
        X_encoded['Departure'], prefix='d'))
    X_encoded = X_encoded.join(
        pd.get_dummies(X_encoded['Arrival'], prefix='a'))
    
    #IATA Data
    iata = pd.read_csv('data/IATA_US_CODES.csv')
    iata['IATA Code']=iata['IATA Code'].astype(str)
    iata['IATA Code']=iata['IATA Code'].str.rstrip()
    iata=iata.set_index('IATA Code')
    iata_dep=iata.add_suffix('_Dep')
    iata_arr=iata.add_suffix('_Arr')
    X_encoded=X_encoded.join(iata_dep, on='Departure', how='left')
    X_encoded=X_encoded.join(iata_arr, on='Arrival', how='left')
    
    #Dummy Iata Dep/Arr
    
    #Dummy Date
    X_encoded['Weekend'] = ((pd.DatetimeIndex(X_encoded['DateOfDeparture']).dayofweek) // 5 == 1).astype(float)
    X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
    X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
    X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
    X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
    X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
    X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
    X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
    X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

    #We one hot encode all those
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))
    X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
    
    
    
    
    return X_encoded



In [281]:
X_train = transform(X_train)
X_train.head()

KeyError: 'Departure'

## Train the model

In [106]:
clf = RandomForestRegressor(
            n_estimators=600, max_depth=110, max_features='sqrt',
             min_samples_split=10, min_samples_leaf=1, bootstrap=True)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=600,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [107]:
y_pred_train = clf.predict(X_train)
score_type = problem.score_types[0]
print("train rmse: ", score_type(y_train, y_pred_train))

train rmse:  0.19603163056158732
